#40 Slice

In [41]:
#in_path = "C:/Users/RAZER/Downloads/Task09_Spleen/dicm_files/labels"

#out_path = "C:/Users/RAZER/Downloads/Task09_Spleen/dicm_groups/labels"

in_path = "C:/Users/RAZER/Downloads/Task09_Spleen/dicm_files/images"
out_path = "C:/Users/RAZER/Downloads/Task09_Spleen/dicm_groups/images"

In [ ]:
!pip install glob2

to moving the files

In [ ]:
!pip install pytest-shutil                    

In [5]:
from glob import glob
import shutil
import os

In [113]:


def move_file(path_in,path_out,move = False,count = 0):
    if count <=0:                                                            #calculating least file count
        size_count = []
        for i in glob(path+"/*"):
            size_count.append(len(glob(i+"/*")))
        count = min(size_count)
    

    for i in glob(path_in + "/*"):
        i_name = os.path.basename(os.path.normpath(i))                       
        num_of_folders = int(len(glob(i+"/*"))/count)                       #calculating number of folders
        print('Total Files In Input_File('+i+"): "+str(len(glob(i+"/*"))))

        for j in range(1,num_of_folders+1):                                 #making new directories
            out_new_dir = os.path.join(path_out, i_name+"_"+str(j))
            os.mkdir(out_new_dir)
            
            k=1
            for file in glob(i+"/*"):
                if k ==count+1:
                    break
                if move == False:
                    shutil.copy(file, out_new_dir)
                else:
                    shutil.move(file,out_new_dir)
                k+=1
            print("Total Files In Output directory("+out_new_dir+") :",k-1)
        print("----------------------------------------------------------------")




In [115]:
path = "C:/Users/RAZER/Downloads/Task09_Spleen/lets_work/dicm_files/images"
path2 = "C:/Users/RAZER/Downloads/Task09_Spleen/lets_work/dicm_groups/images"

move_file(path,path2,True)

Total Files In Input_File(C:/Users/RAZER/Downloads/Task09_Spleen/lets_work/dicm_files/images\img_2): 90
Total Files In Output directory(C:/Users/RAZER/Downloads/Task09_Spleen/lets_work/dicm_groups/images\img_2_1) : 40
Total Files In Output directory(C:/Users/RAZER/Downloads/Task09_Spleen/lets_work/dicm_groups/images\img_2_2) : 40
----------------------------------------------------------------
Total Files In Input_File(C:/Users/RAZER/Downloads/Task09_Spleen/lets_work/dicm_files/images\img_3): 40
Total Files In Output directory(C:/Users/RAZER/Downloads/Task09_Spleen/lets_work/dicm_groups/images\img_3_1) : 40
----------------------------------------------------------------


#Convert dicom to nifties

In [ ]:
!pip install dicom2nifti

In [128]:
import dicom2nifti


def dicom_to_nifty(path_in, path_out):

    file_list = glob(path_in)
    for i in file_list:
        i_name = os.path.basename(os.path.normpath(i))
        dicom2nifti.dicom_series_to_nifti(i,os.path.join(path_out, i_name + ".nii.gz"))


In [133]:
in_path_images = "C:/Users/RAZER/Downloads/Task09_Spleen/lets_work/dicm_groups/images/*"

in_path_labels = "C:/Users/RAZER/Downloads/Task09_Spleen/lets_work/dicm_groups/labels/*"


out_path_images = "C:/Users/RAZER/Downloads/Task09_Spleen/lets_work/niftis_files/images"

out_path_labels = "C:/Users/RAZER/Downloads/Task09_Spleen/lets_work/niftis_files/labels"


dicom_to_nifty(in_path_labels,out_path_labels)
dicom_to_nifty(in_path_images,out_path_images)


#Find Null

In [10]:
import nibabel as nib
import numpy as np

we will see the value of pixels. more then only one value means -> contains labels, 0 for the background and 1 for the forground. only zero means -> only background

In [31]:
path_of = "C:/Users/RAZER/Downloads/Task09_Spleen/lets_work/niftis_files/labels/*"



def find_only_bg(path):
    
    list_of = []
    
    for i in glob(path):
        nifti_file = nib.load(i)

        frame_data =  nifti_file.get_fdata()                            #this giving the infos abour frame

        np_unique = np.unique(frame_data)                               #this will give the unique values of the frame

        if len(np_unique) == 1:
            list_of.append(os.path.basename(os.path.normpath(i)))
            
    return list_of
        
    

In [32]:
non_usable = find_only_bg(path_of)

#PreProcessing

-> cuda <br>
-> cudnn <br>
-> monai <br>
-> pytorch <br>

In [4]:
!pip install -qU "monai[ignite, nibabel, torchvision, tqdm]==0.6.0"

In [7]:
from tqdm import tqdm
from monai.transforms import(
    Compose,
    AddChanneld,
    LoadImaged,
    Resized,
    ToTensord,
    Spacingd,
    Orientationd,
    ScaleIntensityRanged,
    CropForegroundd,
)
from monai.data import DataLoader, Dataset, CacheDataset
from monai.utils import set_determinism

In [ ]:
def prepare(in_dir, pixdim=(1.5, 1.5, 1.0), a_min=-200, a_max=200, spatial_size=[128,128,64], cache=False):

    """
    This function is for preprocessing, it contains only the basic transforms, but you can add more operations that you 
    find in the Monai documentation.
    https://monai.io/docs.html
    """

    set_determinism(seed=0)

    path_train_volumes = sorted(glob(os.path.join(in_dir, "TrainVolumes", "*.nii.gz")))
    path_train_segmentation = sorted(glob(os.path.join(in_dir, "TrainSegmentation", "*.nii.gz")))

    path_test_volumes = sorted(glob(os.path.join(in_dir, "TestVolumes", "*.nii.gz")))
    path_test_segmentation = sorted(glob(os.path.join(in_dir, "TestSegmentation", "*.nii.gz")))

    train_files = [{"vol": image_name, "seg": label_name} for image_name, label_name in zip(path_train_volumes, path_train_segmentation)]
    test_files = [{"vol": image_name, "seg": label_name} for image_name, label_name in zip(path_test_volumes, path_test_segmentation)]

    train_transforms = Compose(
        [
            LoadImaged(keys=["vol", "seg"]),
            AddChanneld(keys=["vol", "seg"]),
            Spacingd(keys=["vol", "seg"], pixdim=pixdim, mode=("bilinear", "nearest")),
            Orientationd(keys=["vol", "seg"], axcodes="RAS"),
            ScaleIntensityRanged(keys=["vol"], a_min=a_min, a_max=a_max, b_min=0.0, b_max=1.0, clip=True), 
            CropForegroundd(keys=["vol", "seg"], source_key="vol"),
            Resized(keys=["vol", "seg"], spatial_size=spatial_size),   
            ToTensord(keys=["vol", "seg"]),

        ]
    )

    test_transforms = Compose(
        [
            LoadImaged(keys=["vol", "seg"]),
            AddChanneld(keys=["vol", "seg"]),
            Spacingd(keys=["vol", "seg"], pixdim=pixdim, mode=("bilinear", "nearest")),
            Orientationd(keys=["vol", "seg"], axcodes="RAS"),
            ScaleIntensityRanged(keys=["vol"], a_min=a_min, a_max=a_max,b_min=0.0, b_max=1.0, clip=True), 
            CropForegroundd(keys=['vol', 'seg'], source_key='vol'),
            Resized(keys=["vol", "seg"], spatial_size=spatial_size),   
            ToTensord(keys=["vol", "seg"]),

            
        ]
    )

    if cache:
        train_ds = CacheDataset(data=train_files, transform=train_transforms,cache_rate=1.0)
        train_loader = DataLoader(train_ds, batch_size=1)

        test_ds = CacheDataset(data=test_files, transform=test_transforms, cache_rate=1.0)
        test_loader = DataLoader(test_ds, batch_size=1)

        return train_loader, test_loader

    else:
        train_ds = Dataset(data=train_files, transform=train_transforms)
        train_loader = DataLoader(train_ds, batch_size=1)

        test_ds = Dataset(data=test_files, transform=test_transforms)
        test_loader = DataLoader(test_ds, batch_size=1)

        return train_loader, test_loader